In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from time import strftime,localtime
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_distances

#Adding this to imports because I kept skipping the cell
scaler = StandardScaler()

In [2]:
!pip install ipython-autotime

In [3]:
%load_ext autotime

time: 511 µs (started: 2021-03-24 20:52:41 -07:00)


In [4]:
#Put this function in front of the 'chunk' cells later (or any cells you
# want to time while they are running).  You don't need to pass anything
# into it.

def st_time():
  now=datetime.now()
  st_time=now.strftime("%H:%M:%S")
  print('Start time:', st_time)
  return

time: 1.13 ms (started: 2021-03-24 20:52:42 -07:00)


# Load in Main Data Frame

In [5]:
#This is the dataframe used from all ~82,000 songs taken from a single day's session.
# Use this data frame to put into the "df.sample(n=)"" function then add the tracer
# album file af_vv_df.

#read in all data from drive
af_tot_df = pd.read_csv('/content/drive/MyDrive/Project5/af_tot_df.csv')
#drop weird column 'Unnamed: 0' that appears
af_tot_df = af_tot_df.drop(columns=['Unnamed: 0'])
#assign track_id to a new column as 'track_id2' and set 'track_id' to index.
af_tot_df['track_idi']=af_tot_df['track_id']
#set index as 'track_id' since it's the unique identifier for every song and can call in all data.
af_tot_df.index=af_tot_df['track_idi']
#show the dataframe to see results
af_tot_df

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Project5/af_tot_df.csv'

time: 246 ms (started: 2021-03-24 20:52:54 -07:00)


# Load in Tracer Tracks

In [ ]:
#Tracer album to be put into library after random sampling.
#read in all data from drive
af_vv_df = pd.read_csv('/content/drive/MyDrive/Project5/af_vv_df.csv')
##drop weird column 'Unnamed: 0' that appears 
af_vv_df = af_vv_df.drop(columns=['Unnamed: 0'])
#assign track_id to a new column as 'track_id2' and set 'track_id' to index.
af_vv_df['track_idi']=af_vv_df['track_id']
#set index as 'track_id' since it's the unique identifier for every song and can call in all data.
af_vv_df.index=af_vv_df['track_idi']
af_vv_df = af_vv_df.drop(columns=['track_idi'])
#show the dataframe to see results
af_vv_df

,track_id,track_name,album_id,album_name,artist_id,artist_name,track_preview_url,track_number,track_href,track_external_urls,track_external_id_isrc,album_href,album_ext_url,album_release_date,album_release_date_precision,album_total_tracks,album_type,album_image_large,album_image_medium,album_image_small,artist_href,artists_ext_url,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,time_signature,track_length,valence
track_idi,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
57lYOdic1C2GVyTPZsJhlB,57lYOdic1C2GVyTPZsJhlB,Necksweat,72yXt1N6iw8TnCG4KeX5pr,Immolate,3hFXoighTNpdkuXcwFu09n,Vaso Vagal,https://p.scdn.co/mp3-preview/dee897818c302af8...,1,https://api.spotify.com/v1/tracks/57lYOdic1C2G...,https://open.spotify.com/track/57lYOdic1C2GVyT...,TCADA1792090,https://api.spotify.com/v1/albums/72yXt1N6iw8T...,https://open.spotify.com/album/72yXt1N6iw8TnCG...,2017-05-12,day,9,album,https://i.scdn.co/image/ab67616d0000b273368290...,https://i.scdn.co/image/ab67616d00001e02368290...,https://i.scdn.co/image/ab67616d00004851368290...,https://api.spotify.com/v1/artists/3hFXoighTNp...,https://open.spotify.com/artist/3hFXoighTNpdku...,0.749000,0.352,0.369,0.929,9,0.1080,-9.028,1,0,0.0445,123.566,4,300176,0.3590
3Rv7s6zTx6KTYZfrpBdW4C,3Rv7s6zTx6KTYZfrpBdW4C,Soar,72yXt1N6iw8TnCG4KeX5pr,Immolate,3hFXoighTNpdkuXcwFu09n,Vaso Vagal,https://p.scdn.co/mp3-preview/2f8fe8e08ca3cdae...,2,https://api.spotify.com/v1/tracks/3Rv7s6zTx6KT...,https://open.spotify.com/track/3Rv7s6zTx6KTYZf...,TCADA1792174,https://api.spotify.com/v1/albums/72yXt1N6iw8T...,https://open.spotify.com/album/72yXt1N6iw8TnCG...,2017-05-12,day,9,album,https://i.scdn.co/image/ab67616d0000b273368290...,https://i.scdn.co/image/ab67616d00001e02368290...,https://i.scdn.co/image/ab67616d00004851368290...,https://api.spotify.com/v1/artists/3hFXoighTNp...,https://open.spotify.com/artist/3hFXoighTNpdku...,0.719000,0.349,0.367,0.423,7,0.0568,-7.647,1,0,0.0430,158.815,4,246319,0.1330
0Qj2eDY1Nku7OGymMs8HGz,0Qj2eDY1Nku7OGymMs8HGz,Cascade,72yXt1N6iw8TnCG4KeX5pr,Immolate,3hFXoighTNpdkuXcwFu09n,Vaso Vagal,https://p.scdn.co/mp3-preview/6f095534525eec32...,3,https://api.spotify.com/v1/tracks/0Qj2eDY1Nku7...,https://open.spotify.com/track/0Qj2eDY1Nku7OGy...,TCADA1792176,https://api.spotify.com/v1/albums/72yXt1N6iw8T...,https://open.spotify.com/album/72yXt1N6iw8TnCG...,2017-05-12,day,9,album,https://i.scdn.co/image/ab67616d0000b273368290...,https://i.scdn.co/image/ab67616d00001e02368290...,https://i.scdn.co/image/ab67616d00004851368290...,https://api.spotify.com/v1/artists/3hFXoighTNp...,https://open.spotify.com/artist/3hFXoighTNpdku...,0.913000,0.496,0.167,0.964,1,0.0853,-8.508,0,0,0.0445,99.872,4,253442,0.2380
2pgLRrQEvDwd6jTXGDswQm,2pgLRrQEvDwd6jTXGDswQm,Ausländer,72yXt1N6iw8TnCG4KeX5pr,Immolate,3hFXoighTNpdkuXcwFu09n,Vaso Vagal,https://p.scdn.co/mp3-preview/f6ec6a950b478ce3...,4,https://api.spotify.com/v1/tracks/2pgLRrQEvDwd...,https://open.spotify.com/track/2pgLRrQEvDwd6jT...,TCADA1792177,https://api.spotify.com/v1/albums/72yXt1N6iw8T...,https://open.spotify.com/album/72yXt1N6iw8TnCG...,2017-05-12,day,9,album,https://i.scdn.co/image/ab67616d0000b273368290...,https://i.scdn.co/image/ab67616d00001e02368290...,https://i.scdn.co/image/ab67616d00004851368290...,https://api.spotify.com/v1/artists/3hFXoighTNp...,https://open.spotify.com/artist/3hFXoighTNpdku...,0.655000,0.318,0.272,0.962,4,0.1050,-11.335,0,0,0.0339,97.206,4,361024,0.0389
5SzbfdSfxmmlyTEZcpb3EH,5SzbfdSfxmmlyTEZcpb3EH,County Fair,72yXt1N6iw8TnCG4KeX5pr,Immolate,3hFXoighTNpdkuXcwFu09n,Vaso Vagal,https://p.scdn.co/mp3-preview/f148a09e067891d4...,5,https://api.spotify.com/v1/tracks/5SzbfdSfxmml...,https://open.spotify.com/track/5SzbfdSfxmmlyTE...,TCADA1792179,https://api.spotify.com/v1/albums/72yXt1N6iw8T...,https://open.spotify.com/album/72yXt1N6iw8TnCG...,2017-05-12,day,9,album,https://i.scdn.co/image/ab67616d0000b273368290...,https://i.scdn.co/image/ab67616d00001e02368290...,https://i.scdn.co/imag

time: 75 ms (started: 2021-03-23 04:28:08 +00:00)


## 9 Tracer tracks so make 9 matrices

In [ ]:
#IN = big dataframe ("df_in"), slice size ("sice_size")
#OUT= matrices 'slice_size' rows long in the form of a list of dataframes

#How? Slicing as functions of input
def matrixSplit(df_in,slice_size):
  df_slices = []                                                      #empty list to add df slices to
  rows_to_split = len(df_in)                                          #counter of len of the rows left to remove via slicing
  while rows_to_split > slice_size:
    df_slices.append(df_in[(rows_to_split-slice_size):rows_to_split]) #append with matrix slice from a counter that will go down by the slice size each iteration
    rows_to_split -= slice_size                                       #this subtracts the slice size from the counter
  else:                                                               #when the conter is less than the slice size
    df_slices.append(df_in[0:rows_to_split])                          #then slice from 0 to remainder
  return df_slices                                                    #return the list of df slices

time: 4.55 ms (started: 2021-03-23 04:28:09 +00:00)


In [ ]:
#Run function to split data in 10,000 track df's.
df_slices = matrixSplit(af_tot_df,10000)

#Seperate list of df's into individual df's
slice_0 = df_slices[0]
slice_1 = df_slices[1]
slice_2 = df_slices[2]
slice_3 = df_slices[3]
slice_4 = df_slices[4]
slice_5 = df_slices[5]
slice_6 = df_slices[6]
slice_7 = df_slices[7]
slice_8 = pd.concat((df_slices[8],af_vv_df),axis=0)

time: 21.4 ms (started: 2021-03-23 04:28:10 +00:00)


In [ ]:
#cos loop

def top_n(df, num_of_results):
  st_time()
  ref = df
  df_list=[]
  for track in df.iloc[:,22:]:
    df = scaler.fit_transform(df.iloc[:,22:]) #skips first 23 columns, which are metadata.
    df = cosine_distances(df)
    df = pd.DataFrame(df)
    df = df[df['track']<1]
    df = track_id.sort_values(by=['track'], ascending=True)
    df = df[:,num_of_results]
    #df = pd.DataFrame(df.values,columns=ref['track_id'],index=ref.index)
    df_list.append(df)
  return df_list

time: 13.1 ms (started: 2021-03-23 02:03:28 +00:00)


In [ ]:
#Function test cell
test = top_n(af_vv_df,99)


Start time: 02:03:29


ValueError: ignored

time: 27.1 ms (started: 2021-03-23 02:03:29 +00:00)


In [ ]:
#cos loop
def top_n(df, num_of_results):
  st_time()
  df_fit_trans = scaler.fit_transform(df.iloc[:,22:])
  ref = df
  df_list=[]
  for col in df:
    df_cos = cosine_distances(row)
    # df_cos = pd.DataFrame(df_cos)
    # df_cos = df_cos[df_cos['track']<1]
    # df_cos = COLJUMN.sort_values(by=['track'], ascending=True)
    # df_cos = df_cos[:,num_of_results]
    # #df = pd.DataFrame(df.values,columns=ref['track_id'],index=ref.index)
    df_list.append(df_cos)
  return df_list

time: 7.24 ms (started: 2021-03-23 02:59:43 +00:00)


In [ ]:
#Function test cell
test = top_n(af_vv_df,99)


Start time: 02:59:55


ValueError: ignored

time: 25.4 ms (started: 2021-03-23 02:59:55 +00:00)


In [ ]:
af_vv_df_fit_trans = scaler.fit_transform(af_vv_df.iloc[:,22:]) 


#af_vv_df_ft = scaler.fit_transform(af_vv_df.iloc[:,22:]) #skips first 23 columns, which are metadata.

time: 6.57 ms (started: 2021-03-23 04:28:29 +00:00)


In [ ]:
#This is the part Anubhav helped me with.  I was having issues getting
# sort in the right indent.  I was too far intended relative to the for loop
# so that it was not doing anything.

num_rows,num_cols = cos_df.shape
top_three = []
for row in range(0,num_rows):
    temp_list = []
    for col in range(0,num_cols):
        if cos_df[row][col] < 1.5:
            temp_list.append(cos_df[row][col])
    temp_list.sort(reverse=True)
    top_three.append(temp_list[0:3])
print('Length of top_three is: ', len(top_three))
top_three

Length of top_three is:  9


[[1.4448865541065419, 1.4035204175814977, 1.3255637852236417],
 [1.3219183378522485, 1.1922592692371945, 1.1832920561857956],
 [1.4548588967698255, 1.4499959056674077, 1.361859833825495],
 [1.4035204175814977, 1.3781742611122385, 1.250417376579381],
 [1.4499959056674077, 1.3781742611122385, 1.3338963861694952],
 [1.4448865541065419, 1.361859833825495, 1.3219183378522485],
 [1.3338963861694952, 1.2494238541620233, 1.2316599554936005],
 [1.4548588967698255, 1.1879660589199157, 1.0681290364057632],
 [1.2227949930237239, 1.1929290510622055, 1.1922592692371945]]

time: 15 ms (started: 2021-03-23 05:37:58 +00:00)
